In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import XLNetTokenizer, XLNetModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# 1. Mapping Label
sent_map   = {'Netral': 0, 'Positif': 1, 'Negatif': 2}
aspect_map = {'Pelayanan': 0, 'Kualitas': 1, 'Harga': 2}
inv_sent_map   = {v: k for k, v in sent_map.items()}
inv_aspect_map = {v: k for k, v in aspect_map.items()}

# 2. Dataset Class
class TokopediaDualDataset(Dataset):
    def __init__(self, texts, sent_labels, aspect_labels, tokenizer, max_length=128):
        self.texts = texts
        self.sent_labels = sent_labels
        self.aspect_labels = aspect_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'sentiment': torch.tensor(self.sent_labels[idx], dtype=torch.long),
            'aspect': torch.tensor(self.aspect_labels[idx], dtype=torch.long)
        }

# 3. XLNet-CNN Model
class XLNetCNNMultiTask(nn.Module):
    def __init__(self, num_sent_classes=3, num_aspect_classes=3):
        super().__init__()
        self.xlnet = XLNetModel.from_pretrained("xlnet-base-cased")
        for param in self.xlnet.parameters():
            param.requires_grad = False
        hidden_size = self.xlnet.config.hidden_size  # 768
        self.conv1 = nn.Conv1d(in_channels=hidden_size, out_channels=300, kernel_size=5)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.sent_classifier = nn.Linear(300, num_sent_classes)
        self.aspect_classifier = nn.Linear(300, num_aspect_classes)

    def forward(self, input_ids, attention_mask):
        out = self.xlnet(input_ids=input_ids, attention_mask=attention_mask)
        x = out.last_hidden_state.permute(0, 2, 1)  # (B, H, T)
        x = self.relu(self.conv1(x))                # (B, F, T_new)
        x = self.pool(x).squeeze(-1)                # (B, F)
        x = self.dropout(x)
        sent_logits = self.sent_classifier(x)
        aspect_logits = self.aspect_classifier(x)
        return sent_logits, aspect_logits

# 4. Load Data
df = pd.read_csv("Tokopedia_preprocessed.csv")
texts = df["clean_content"].tolist()
sent_labels = [sent_map[s] for s in df["Sentimen"]]
aspect_labels = [aspect_map[a] for a in df["Aspek"]]

# 5. Split Data
train_texts, test_texts, train_sent, test_sent, train_aspect, test_aspect = train_test_split(
    texts, sent_labels, aspect_labels, test_size=0.2, random_state=42, stratify=sent_labels
)

# 6. Tokenizer dan DataLoader
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
train_dataset = TokopediaDualDataset(train_texts, train_sent, train_aspect, tokenizer)
test_dataset = TokopediaDualDataset(test_texts, test_sent, test_aspect, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# 7. Training Preparation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XLNetCNNMultiTask().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 8. Train Loop
for epoch in range(10):
    model.train()
    total_loss, correct_sent, correct_aspect = 0, 0, 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        sent_labels = batch["sentiment"].to(device)
        aspect_labels = batch["aspect"].to(device)

        optimizer.zero_grad()
        sent_logits, aspect_logits = model(input_ids, attention_mask)
        loss = criterion(sent_logits, sent_labels) + criterion(aspect_logits, aspect_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct_sent += (sent_logits.argmax(1) == sent_labels).sum().item()
        correct_aspect += (aspect_logits.argmax(1) == aspect_labels).sum().item()

    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f} - Sent Acc: {correct_sent/len(train_dataset):.4f} - Asp Acc: {correct_aspect/len(train_dataset):.4f}")

# 9. Evaluation
model.eval()
all_sent_preds, all_sent_trues = [], []
all_asp_preds, all_asp_trues = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        sent_labels = batch["sentiment"].to(device)
        aspect_labels = batch["aspect"].to(device)

        sent_logits, aspect_logits = model(input_ids, attention_mask)
        all_sent_preds.extend(sent_logits.argmax(1).cpu().tolist())
        all_sent_trues.extend(sent_labels.cpu().tolist())
        all_asp_preds.extend(aspect_logits.argmax(1).cpu().tolist())
        all_asp_trues.extend(aspect_labels.cpu().tolist())

# 10. Metrics
print("=== SENTIMEN CLASSIFICATION REPORT ===")
print(classification_report(all_sent_trues, all_sent_preds, target_names=[inv_sent_map[i] for i in range(3)]))

print("=== ASPEK CLASSIFICATION REPORT ===")
print(classification_report(all_asp_trues, all_asp_preds, target_names=[inv_aspect_map[i] for i in range(3)]))

print("=== CONFUSION MATRIX SENTIMEN ===")
print(confusion_matrix(all_sent_trues, all_sent_preds))

print("=== CONFUSION MATRIX ASPEK ===")
print(confusion_matrix(all_asp_trues, all_asp_preds))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

Epoch 1 - Loss: 512.9115 - Sent Acc: 0.4085 - Asp Acc: 0.7125
Epoch 2 - Loss: 406.0443 - Sent Acc: 0.4333 - Asp Acc: 0.7652
Epoch 3 - Loss: 394.3075 - Sent Acc: 0.4444 - Asp Acc: 0.7850
Epoch 4 - Loss: 388.1797 - Sent Acc: 0.4556 - Asp Acc: 0.7880
Epoch 5 - Loss: 386.5627 - Sent Acc: 0.4605 - Asp Acc: 0.7859
Epoch 6 - Loss: 380.0102 - Sent Acc: 0.4715 - Asp Acc: 0.7897
Epoch 7 - Loss: 373.7106 - Sent Acc: 0.4871 - Asp Acc: 0.7895
Epoch 8 - Loss: 369.1284 - Sent Acc: 0.4815 - Asp Acc: 0.7979
Epoch 9 - Loss: 369.5732 - Sent Acc: 0.4864 - Asp Acc: 0.7981
Epoch 10 - Loss: 366.9814 - Sent Acc: 0.4844 - Asp Acc: 0.7933
=== SENTIMEN CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

      Netral       0.53      0.45      0.49       561
     Positif       0.59      0.73      0.65       767
     Negatif       0.59      0.51      0.55       672

    accuracy                           0.58      2000
   macro avg       0.57      0.56      0.56      2000
weighted avg  